<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC1_Turbofan_FD002_Model_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures

In [3]:
sns.set_palette('colorblind')

# Data Preparation

In [4]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [5]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [6]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [7]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(2)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,20.0037,0.7000,100.0,491.19,608.79,1495.60,1269.51,9.35,...,314.05,2389.02,8169.64,9.3035,0.03,369,2324,100.00,24.36,14.5189
53755,260,313,10.0022,0.2510,100.0,489.05,605.81,1514.32,1324.12,10.52,...,371.22,2388.42,8245.36,8.7586,0.03,374,2319,100.00,28.10,16.9454
53756,260,314,25.0041,0.6200,60.0,462.54,537.48,1276.24,1057.92,7.05,...,163.74,2030.33,7971.25,11.0657,0.02,310,1915,84.93,14.19,8.5503
53757,260,315,25.0033,0.6220,60.0,462.54,537.84,1272.95,1066.30,7.05,...,164.37,2030.35,7972.47,11.0537,0.02,311,1915,84.93,14.05,8.3729


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [8]:
test.shape, y_test.shape

((33991, 26), (259, 1))

In [9]:
test_last = proc.transform_test_keep_setting(test)
test_last.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,10.0076,0.2501,100.0,489.05,605.42,1515.00,1325.07,10.52,15.50,393.58,...,370.87,2388.32,8167.06,8.7456,0.03,371,2319,100.0,28.30,17.0934
1,0.0018,0.0000,100.0,518.67,642.67,1591.67,1418.17,14.62,21.61,553.36,...,521.10,2388.12,8138.12,8.4248,0.03,393,2388,100.0,38.82,23.3463
2,35.0015,0.8412,100.0,449.44,555.86,1370.62,1135.59,5.48,8.00,194.58,...,183.11,2388.07,8071.23,9.3094,0.02,332,2223,100.0,14.75,8.9589
3,20.0032,0.7000,100.0,491.19,607.99,1487.94,1257.49,9.35,13.66,334.39,...,314.88,2388.12,8062.39,9.2349,0.02,365,2324,100.0,24.22,14.6814
4,42.0055,0.8400,100.0,445.00,550.81,1358.95,1140.34,3.91,5.72,138.42,...,130.82,2389.06,8140.94,9.3964,0.02,333,2212,100.0,10.34,6.3601


In [10]:
X_test = test_last

### Remaining Useful Life (RUL)

In [11]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,148
1,1,2,147
2,1,3,146
3,1,4,145
4,1,5,144


## Attributes and target separation

In [12]:
X_train, y_train = proc.X_y_train_divide_with_settings(train)

In [13]:
y_train.head()

,RUL
0,148
1,147
2,146
3,145
4,144


In [14]:
X_train.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


## Training and Evaluation functions

In [15]:
eval = Evaluation()

In [16]:
search = HyperparameterSearch()

# DecisionTreeRegressor

## Linear RUL

In [27]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])

In [28]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'best', 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 20, 'tree_reg__min_impurity_decrease': 3.5, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 200, 'tree_reg__criterion': 'friedman_mse', 'tree_reg__ccp_alpha':

Best Model:
```
DecisionTreeRegressor(ccp_alpha=0.675,
                        criterion='friedman_mse', max_depth=200,
                        max_features='log2',
                        min_impurity_decrease=3.5,
                        min_samples_leaf=20, random_state=42))
```

In [29]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=0.675,
                                       criterion='friedman_mse', max_depth=200,
                                       max_features='log2',
                                       min_impurity_decrease=3.5,
                                       min_samples_leaf=20, random_state=42))
])
model

Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.675,
                                       criterion='friedman_mse', max_depth=200,
                                       max_features='log2',
                                       min_impurity_decrease=3.5,
                                       min_samples_leaf=20, random_state=42))])

In [30]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.577,RMSE=-44.984
(CV) R2=0.526,RMSE=-47.543


In [31]:
eval.show_result(y_test, model.predict(X_test))

R2=0.589,RMSE=-34.462


Poor results. A little better than FD001, which should be easier.

## Non-linear RUL

In [32]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [33]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}, 'trf_reg__regressor__splitter': 'best', 'trf_reg__regressor__min_samples_split': 2, 'trf_reg__regressor__min_samples_leaf': 10, 'trf_reg__regressor__min_impurity_decrease': 5.5, 'trf_reg__regressor__ma

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}}

DecisionTreeRegressor(ccp_alpha=0.375,
                    criterion='friedman_mse',
                    max_depth=200,
                    max_features='sqrt',
                    min_impurity_decrease=5.5,
                    min_samples_leaf=10,
                    random_state=42),
```



In [34]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=0.375,
                            criterion='friedman_mse',
                            max_depth=200,
                            max_features='sqrt',
                            min_impurity_decrease=5.5,
                            min_samples_leaf=10,
                            random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':98})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=0.375,
                                                                            criterion='friedman_mse',
                                                                            max_depth=200,
                                                                            max_features='sqrt',
                                                                            min_impurity_decrease=5.5,
                                                                            min_samples_leaf=10,
                                                                            random_state=42),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f134321ac20>,
                                                                            kw_args={'a_max': 98,
                

In [35]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.750,RMSE=-15.734
(CV) R2=0.727,RMSE=-16.448


In [36]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.800,RMSE=-15.532


The best result we had so far in training and in testing. Very same model as FD001 and in FD003, again!.

## PolyFeatures + Linear RUL

In [37]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [38]:
# ~3min
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": [None, "sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'random', 'tree_reg__min_samples_split': 5, 'tree_reg__min_samples_leaf': 15, 'tree_reg__min_impurity_decrease': 5.0, 'tree_reg__max_features': None, 'tree_reg__max_depth': 300, 'tree_reg__criterion': 'squared_error', 'tree_reg__ccp_alpha'

Best Model:
```
PolynomialFeatures(degree=3, include_bias=False)),
DecisionTreeRegressor(ccp_alpha=0.25, max_depth=300,
                        min_impurity_decrease=5.0,
                        min_samples_leaf=15, min_samples_split=5,
                        random_state=42, splitter='random'))
```


In [39]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=0.25, max_depth=300,
                        min_impurity_decrease=5.0,
                        min_samples_leaf=15, min_samples_split=5,
                        random_state=42, splitter='random'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.25, max_depth=300,
                                       min_impurity_decrease=5.0,
                                       min_samples_leaf=15, min_samples_split=5,
                                       random_state=42, splitter='random'))])

In [40]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.554,RMSE=-46.183
(CV) R2=0.529,RMSE=-47.379


In [41]:
eval.show_result(y_test, model.predict(X_test))

R2=0.633,RMSE=-32.560


No clear benefit from polynomial features in linear RUL, as with the other models. Actually, it worsened the results.

## PolyFeatures + Non-Linear RUL

In [42]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [43]:
# ~3min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": [None, "sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 77}
PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
DecisionTreeRegressor(ccp_alpha=1.2,
                        criterion='friedman_mse',
                        min_impurity_decrease=2.5,
                        min_samples_leaf=20,
                        min_samples_split=5,
                        random_state=42,
                        splitter='random'),
```


In [44]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=1.2,
                                            criterion='friedman_mse',
                                            min_impurity_decrease=2.5,
                                            min_samples_leaf=20,
                                            min_samples_split=5,
                                            random_state=42,
                                            splitter='random'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':77})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=1.2,
                                                                            criterion='friedman_mse',
                                                                            min_impurity_decrease=2.5,
                                                                            min_samples_leaf=20,
                                                                            min_samples_split=5,
                                                                            random_state=42,
                                                                            splitter='random'),
                                            transformer=FunctionTransform

In [45]:
# ~3min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.771,RMSE=-11.104
(CV) R2=0.759,RMSE=-11.393


In [46]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.794,RMSE=-12.161


This time, it improved a little bit the performance.

# RandomForestRegressor

## Linear RUL

In [47]:
model = Pipeline([
    ('tree_reg'  ,   RandomForestRegressor(random_state=42, n_jobs=-1))
])

In [48]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [49]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "tree_reg__n_estimators": [50,100,200],
        "tree_reg__criterion": ['squared_error','absolute_error','poisson'],
        "tree_reg__max_depth": [None,50, 100, 150],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "tree_reg__oob_score": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__oob_score': True, 'tree_reg__n_estimators': 200, 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 15, 'tree_reg__min_impurity_decrease': 0.6, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 50, 'tree_reg__criterion': 'squared_

Best Model:
```
RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                        max_features='log2',
                        min_impurity_decrease=0.6,
                        min_samples_leaf=15, n_estimators=200,
                        n_jobs=-1, oob_score=True,
                        random_state=42))
```

In [50]:
model = Pipeline([
    ('tree_reg'  ,   RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                            max_features='log2',
                            min_impurity_decrease=0.6,
                            min_samples_leaf=15, n_estimators=200,
                            n_jobs=-1, oob_score=True,
                            random_state=42))
])
model

Pipeline(steps=[('tree_reg',
                 RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                                       max_features='log2',
                                       min_impurity_decrease=0.6,
                                       min_samples_leaf=15, n_estimators=200,
                                       n_jobs=-1, oob_score=True,
                                       random_state=42))])

In [51]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.633,RMSE=-41.890
(CV) R2=0.598,RMSE=-43.801


In [52]:
eval.show_result(y_test, model.predict(X_test))

R2=0.695,RMSE=-29.683


A little better than linear regression. It's the very same model as in FD001 and in FD003.

## Non-linear RUL

In [53]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(random_state=42, n_jobs=-1),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [54]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__n_estimators": [50,100,200],
        "trf_reg__regressor__criterion": ['squared_error','absolute_error','poisson'],
        "trf_reg__regressor__max_depth": [None,50, 100, 150],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "trf_reg__regressor__oob_score": [True, False]
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 90}, 'trf_reg__regressor__oob_score': True, 'trf_reg__regressor__n_estimators': 200, 'trf_reg__regressor__min_samples_split': 10, 'trf_reg__regressor__min_samples_leaf': 15, 'trf_reg__regressor__min_impurit

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 90}}

RandomForestRegressor(ccp_alpha=0.975,
                        max_depth=150,
                        max_features='log2',
                        min_impurity_decrease=1.2,
                        min_samples_leaf=15,
                        min_samples_split=10,
                        n_estimators=200,
                        n_jobs=-1,
                        oob_score=True,
                        random_state=42),
```



In [55]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(ccp_alpha=0.975,
                                            max_depth=150,
                                            max_features='log2',
                                            min_impurity_decrease=1.2,
                                            min_samples_leaf=15,
                                            min_samples_split=10,
                                            n_estimators=200,
                                            n_jobs=-1,
                                            oob_score=True,
                                            random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':90})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=RandomForestRegressor(ccp_alpha=0.975,
                                                                            max_depth=150,
                                                                            max_features='log2',
                                                                            min_impurity_decrease=1.2,
                                                                            min_samples_leaf=15,
                                                                            min_samples_split=10,
                                                                            n_estimators=200,
                                                                            n_jobs=-1,
                                                                            oob_score=True,
                                                       

In [56]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.798,RMSE=-12.725
(CV) R2=0.789,RMSE=-13.012


In [57]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.839,RMSE=-12.769


Following the results from the decision trees, we have a good result and good generalization.

## PolyFeatures + Linear RUL

In [58]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   RandomForestRegressor(random_state=42, n_jobs=-1)),
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [60]:
# ~1h
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "tree_reg__n_estimators": [50,100,200],
        "tree_reg__criterion": ['squared_error','absolute_error','poisson'],
        "tree_reg__max_depth": [None, 50, 100, 150],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "tree_reg__oob_score": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__oob_score': True, 'tree_reg__n_estimators': 200, 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 10, 'tree_reg__min_impurity_decrease': 1.1, 'tree_reg__max_features': 'sqrt', 'tree_reg__max_depth': 150, 'tree_reg__criterion': 'squared

Best Model:
```
PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
RandomForestRegressor(ccp_alpha=1.35, max_depth=150,
                        max_features='sqrt',
                        min_impurity_decrease=1.1,
                        min_samples_leaf=10, n_estimators=200,
                        n_jobs=-1, oob_score=True,
                        random_state=42))
```


In [61]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('tree_reg'  ,   RandomForestRegressor(ccp_alpha=1.35, max_depth=150,
                                       max_features='sqrt',
                                       min_impurity_decrease=1.1,
                                       min_samples_leaf=10, n_estimators=200,
                                       n_jobs=-1, oob_score=True,
                                       random_state=42))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('tree_reg',
                 RandomForestRegressor(ccp_alpha=1.35, max_depth=150,
                                       max_features='sqrt',
                                       min_impurity_decrease=1.1,
                                       min_samples_leaf=10, n_estimators=200,
                                       n_jobs=-1, oob_score=True,
                                       random_state=42))])

# Continue here

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.731,RMSE=-51.276
(CV) R2=0.581,RMSE=-62.471


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.314,RMSE=-47.460


No clear benefit from polynomial features in linear RUL, as with the other models. Actually, it worsened the results.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(random_state=42, n_jobs=-1),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__n_estimators": [50,100,200],
        "trf_reg__regressor__criterion": ['squared_error','absolute_error','poisson'],
        "trf_reg__regressor__max_depth": [None, 50, 100, 150],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "trf_reg__regressor__oob_score": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 99}, 'trf_reg__regressor__oob_score': True, 'trf_reg__regressor__n_estimators': 200, 'trf_reg__regressor__min_samples_split': 2, 'trf_reg__regressor__min_samples_leaf': 5, 'trf_reg__regressor__min_impurity_decrease': 5.6, 'trf_reg__regressor__max_features': 'log2', 'trf_reg__regressor__max_depth': 50, 'trf_reg__regressor__c

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 99}
PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)
RandomForestRegressor(ccp_alpha=0.15,
                    max_depth=50,
                    max_features='log2',
                    min_impurity_decrease=5.6,
                    min_samples_leaf=5,
                    n_estimators=200,
                    n_jobs=-1,
                    oob_score=True,
                    random_state=42),
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(ccp_alpha=0.15,
                        max_depth=50,
                        max_features='log2',
                        min_impurity_decrease=5.6,
                        min_samples_leaf=5,
                        n_estimators=200,
                        n_jobs=-1,
                        oob_score=True,
                        random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':99})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=RandomForestRegressor(ccp_alpha=0.15,
                                                                            max_depth=50,
                                                                            max_features='log2',
                                                                            min_impurity_decrease=5.6,
                                                                            min_samples_leaf=5,
                                                                            n_estimators=200,
                                                                            n_jobs=-1,
                                                                            oob_score=True,
 

In [ ]:
# ~3min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.848,RMSE=-11.880
(CV) R2=0.830,RMSE=-12.395


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.776,RMSE=-15.112


Exactly the same model as FD001.

Same results as the decision tree. Considering that the tree is quicker to train, one should choose the tree over the forest. It also allows us to use the tree as feature selector to other models.